# NAF / ROME Link Factory
* Utiliser les diverses données pour calculer une table de correspondance rome / naf


## Autres idées non exploitées:
- pousser certains domaines naf manuellement reliés aux romes
- exploiter les codes OGR (données insuffisantes)
- obtenir des données plus larges, sur une pluse grande durée (cf DPAE)
- intégrer nombre d'entreprises et leur taille moyenne (sur tout le territoire) (cf. sirene/insee)
- ...

In [127]:
## Chargement des labels naf, rome, ogr
from IPython.display import display, HTML
import pandas as pd
import numpy as np

# NAF
naf_labels = pd.read_csv('../ressources/list_NAF_LBB.csv', sep='|', encoding="utf-8")
naf_labels.columns = ['nafdot', 'naf', 'label']
print(f"Obtained {len(naf_labels)} NAF labels")
# display(HTML(naf_labels.head(5).to_html()))

# ROME
rome_labels = pd.read_csv('../ressources/liste_rome_LBB.csv', sep=',', encoding="utf-8")
rome_labels.columns = ['rome', 'rome_1', 'rome_2', 'rome_3', 'label', 'slug']
print(f"Obtained {len(rome_labels)} ROME labels")
# display(HTML(rome_labels.head(5).to_html()))

# Chargement des statistiques d'emploi
emploi_rome_naf = pd.read_csv('../ressources/contrats_30j.csv', sep=',', encoding="utf-8")[['ROME', 'APE700', 'nb_embauches']]
emploi_rome_naf.columns = ['rome', 'naf', 'embauches']
# display(HTML(emploi_rome_naf.head(5).to_html()))
        
    
# Calcul des ratios
naf_embauches = emploi_rome_naf[['naf', 'embauches']].groupby('naf').agg(
    embauches_total_n=pd.NamedAgg(column='embauches', aggfunc=sum)
)
rome_embauches = emploi_rome_naf[['rome', 'embauches']].groupby('rome').agg(
    embauches_total_r=pd.NamedAgg(column='embauches', aggfunc=sum)
)


naf_i = (
emploi_rome_naf
    .merge(naf_embauches, on='naf')
    .merge(rome_embauches, on='rome')
)
naf_i['ratio_naf'] = ( naf_i.embauches / naf_i.embauches_total_n ) * 100
naf_i['ratio_rome'] = ( naf_i.embauches / naf_i.embauches_total_r ) * 100

naf_i = naf_i.sort_values(by=['ratio_naf'], ascending=False)



Obtained 732 NAF labels
Obtained 531 ROME labels


In [136]:
# Compute best nafs for rome
ROME = 'M1705'

print(f"Table ROME=>NAF pour \"{rome_labels.loc[(rome_labels['rome'] == ROME, 'label')].iloc[0]}\"")

# Basic order based on employment percentage
# Use rome recruitment % by naf ratio, stop @ 75% of total rome pool
df_base = (
naf_i.loc[naf_i.rome == ROME]
    .sort_values(by=['ratio_naf'], ascending=False)
)
df_base['cumsum'] =  100 - df_base['ratio_rome'].cumsum()
df_base = df_base.loc[df_base['cumsum'] > 25]
df_base['score'] = pd.to_numeric(pd.cut(
    df_base['ratio_naf'],
    bins=[1, 2, 5, 10, 50, 100],
    labels=[0, 1, 2, 3, 4]
))

# Boosting scores with rome orientations
# If naf is in top 10 of rome destinations, increment score
df_rs = (
naf_i.loc[naf_i.rome == ROME ]
    .sort_values(by=['ratio_rome'], ascending=False)
)
df_rs['cumsum'] =  100 - df_rs['ratio_rome'].cumsum()
df_rs = df_rs.head(10)
df_base.loc[df_base['naf'].isin(df_rs['naf']), 'score'] += 1
df_base = df_base.loc[df_base['score'].ge(1)]

# Display
(
df_base
    .merge(naf_labels, on='naf')
    .sort_values(by=['score'], ascending=False)
    [['rome', 'naf', 'label','score']]
).head(15)

Table ROME=>NAF pour "Marketing"


,rome,naf,label,score
0,M1705,6312Z,Portails Internet,2.0
42,M1705,6202A,Conseil en systèmes et logiciels informatiques,2.0
28,M1705,7311Z,Activités des agences de publicité,2.0
30,M1705,7022Z,Conseil pour les affaires et autres conseils d...,2.0
9,M1705,6201Z,Programmation informatique,2.0
1,M1705,5913B,Édition et distribution vidéo,2.0
41,M1705,8421Z,Affaires étrangères,1.0
40,M1705,8560Z,Activités de soutien à l'enseignement,1.0
26,M1705,5814Z,Édition de revues et périodiques,1.0
27,M1705,5829A,Édition de logiciels système et de réseau,1.0


In [ ]:
# Legacy Code

# df_ok['score'] = 0
# df_ok.loc[df_cumsum < 15, 'score'] = 5
# df_ok['score'] = pd.cut(
 #   df_ok['cumsum'],
 #   bins=[0, 15, 30, 45, 60, 100],
 #   labels=[5, 4, 3, 2, 1]
#)
#df_ok

#df_base['score'] = 0
#df_base.loc[df_base['cumsum'].ge(0), 'score'] = 0
#df_base.loc[df_base['cumsum'].ge(15).lt(30), 'score'] = 1
#df_base.loc[df_base['cumsum'].ge(30).lt(45), 'score'] = 2
#df_base.loc[df_base['cumsum'].ge(45).lt(60), 'score'] = 3
#df_base.loc[df_base['cumsum'].ge(60), 'score'] = 4
#df_base